In [1]:
import numpy as np

In [ ]:
def lorenz_ekf(x):
    rho = 28
    beta = 8/3
    sigma = 10
    T = 100
    n = 100000
    dt = T/n
    M = 5

    Sigma1 = np.array([[1]])

    x_vals = x.copy()

    Sigma2 = np.diag([.1,0])   

    # random starting point
    u0 = 2*np.random.randn(2)
    u_hat = np.zeros((2,n))
    u_hat[:,0] = u0

    R = np.eye(2)
    Sig_inv = np.linalg.inv(Sigma1@Sigma1.T)

    u_ens = np.zeros((2,M,n))
    for j in range(M):
       u_ens[:,j,0] = u0 + 0.1*np.random.randn(2)

    for i in range(1, n): 
        for j in range(M):

            A0 = -sigma*x[i]
            A1 = np.array([[sigma,0]])

            a1 = np.array([[-1,-x_vals[i]],[x_vals[i],-beta]])
            a0 = np.array([rho*x_vals[i],0])

            duI = x_vals[i] - x_vals[i-1]

            innovation = duI - (A0 + (A1 @ u_ens[:,i-1])) * dt
            u_ens[:,i] = u_ens[:,i-1] + dt*(a1@u_ens[:,i-1] + a0) + ((R @ A1.T) @ Sig_inv * innovation).flatten()
            R += (a1@R + R@ a1.T  + Sigma2@Sigma2.T - R@A1.T@Sig_inv@A1@R)*dt

            if np.isnan(R).any():
                print(f"NaN detected at step {i}")
                break
    return u_ens




